In [6]:
import geopandas as gpd
import pandas as pd
import fiona
import os

In [34]:
import pandas as pd
import geopandas as gpd

# File paths
oa_population_shapefile = './data/ABPOGB_CSV/output_area_population.shp'
oa_age_bands_csv = './data/oa_age_bands.csv'
national_park_pop_csv = './data/national_park_pop.csv'

# Load shapefile and CSVs
print("Loading shapefile and CSVs...")
oa_gdf = gpd.read_file(oa_population_shapefile)
oa_age_bands_df = pd.read_csv(oa_age_bands_csv)
national_park_pop_df = pd.read_csv(national_park_pop_csv)

# Ensure population CSV column names are consistent
oa_age_bands_df.columns = ['OA21CD', 'Population']
national_park_pop_df.columns = ['National_Park', '2011_total', '2021_total']

# Keep only 2021 data and drop rows with 'n/a' in 2021_total
national_park_pop_df = national_park_pop_df.drop(columns=['2011_total'])
national_park_pop_df = national_park_pop_df.dropna(subset=['2021_total'])

# Convert '2021_total' to integer
national_park_pop_df['2021_total'] = national_park_pop_df['2021_total'].str.replace(',', '').astype(int)

# Merge the output area population data with the OA GeoDataFrame
print("Merging output area population data with OA GeoDataFrame...")
oa_gdf = oa_gdf.merge(oa_age_bands_df, on='OA21CD', how='left')

# Print columns of oa_gdf to verify
print("Columns in oa_gdf:")
print(oa_gdf.columns)

# Print the first few rows of oa_gdf to inspect the data
print("First few rows of oa_gdf:")
print(oa_gdf.head())

# Identify the correct column for the population
population_column = 'Population_y' if 'Population_y' in oa_gdf.columns else 'Population'

# Calculate the population per address
print("Calculating population per address...")
oa_gdf['pop_per_address'] = oa_gdf[population_column] / oa_gdf['total_addr']

# Calculate the population inside and outside the national parks
oa_gdf['pop_inside'] = oa_gdf['pop_per_address'] * oa_gdf['address_in']
oa_gdf['pop_outside'] = oa_gdf['pop_per_address'] * oa_gdf['address_ou']

# Define the threshold for the percentage of population inside the national park
threshold = 0.90

# Filter the output areas based on the threshold
oa_gdf['percentage_inside'] = oa_gdf['pop_inside'] / oa_gdf[population_column]
filtered_oa_gdf = oa_gdf[oa_gdf['percentage_inside'] >= threshold]

# Group by national park to get the total population inside each park
print("Grouping by national park to get total population inside each park...")
national_park_population = filtered_oa_gdf.groupby('National_P').agg({
    'pop_inside': 'sum'
}).reset_index()

# Merge the national park total population from CSV
national_park_population = national_park_population.merge(national_park_pop_df, left_on='National_P', right_on='National_Park', how='left')

# Calculate the percentage of the population inside the national park compared to the total population
national_park_population['percentage_inside'] = national_park_population['pop_inside'] / national_park_population['2021_total'] * 100

# Display the result
print(national_park_population)

# Save the result to a new CSV file
output_csv = '../data/ABPOGB_CSV/national_park_population_analysis.csv'
print(f"Saving results to {output_csv}...")
national_park_population.to_csv(output_csv, index=False)
print("Script completed successfully!")


Loading shapefile and CSVs...


DataSourceError: ../data/ABPOGB_CSV/output_area_population.shp: No such file or directory